# Picking Algorithms

In this exercise you get code snippets for two optimizations that fail silently. You need to fix them by selecting a different optimization algorithm.

If you find one that works, continue to search for one that works better. 

In [33]:
import numpy as np
import estimagic as em

## Problem 1

This is a modified version of the `powell_singular` function from the More and Wild set of test functions. 

In [37]:
def powell_steps(x):
    x = x.round(4)
    fvec = np.zeros(4)
    fvec[0] = x[0] + 10 * x[1]
    fvec[1] = np.sqrt(5.0) * (x[2] - x[3])
    fvec[2] = (x[1] - 2 * x[2]) ** 2
    fvec[3] = np.sqrt(10.0) * (x[0] - x[3]) ** 2
    out = {"root_contributions": fvec, "value": fvec @ fvec}
    return out

powell_start = np.array([3, -1, 0, 1])
# correct parameters
powell_params = np.zeros(4)
# correct criterion value
powell_criterion = 0

In [38]:
res_powell = em.minimize(
    criterion=powell_steps,
    params=powell_start,
    algorithm="scipy_lbfgsb",
)
res_powell.params

array([ 3., -1.,  0.,  1.])

The optimization thinks it terminated "successfully" but when you look at `res.params` you see that it actually got stuck at the start values. 

## Problem 2

This is the `bratu_2d` problem from the Cartis and Roberts benchmark set. 

In [42]:
def bratu(x):
    alpha = 4.
    x = x.reshape((int(np.sqrt(len(x))), int(np.sqrt(len(x)))))
    p = x.shape[0] + 2
    h = 1 / (p - 1)
    c = h**2 * alpha
    xvec = np.zeros((x.shape[0] + 2, x.shape[1] + 2))
    xvec[1 : x.shape[0] + 1, 1 : x.shape[1] + 1] = x
    fvec = np.zeros_like(x)
    for i in range(2, p):
        for j in range(2, p):
            fvec[i - 2, j - 2] = (
                4 * xvec[i - 1, j - 1]
                - xvec[i, j - 1]
                - xvec[i - 2, j - 1]
                - xvec[i - 1, j]
                - xvec[i - 1, j - 2]
                - c * np.exp(xvec[i - 1, j - 1])
            )
    fvec = fvec.flatten()
    out = {"root_contributions": fvec, "value": fvec @ fvec}
    return out

bratu_start = np.zeros(64)
# correct criterion value
bratu_criterion = 0
# skip correct params because they are too long

In [43]:
res_bratu = em.minimize(
    criterion=bratu,
    params=bratu_start,
    algorithm="scipy_neldermead",
    algo_options={"stopping.max_criterion_evaluations": 5000},
)

res_bratu

Minimize with 64 free parameters terminated unsuccessfully after 4000 criterion evaluations and 3781 iterations.

The value of criterion improved from 0.1560737692424935 to 0.14641530058365018.

The scipy_neldermead algorithm reported: Maximum number of function evaluations has been exceeded.

Independent of the convergence criteria used by scipy_neldermead, the strength of convergence can be assessed by the following criteria:

                             one_step     five_steps 
relative_criterion_change  0.0003009      0.001361   
relative_params_change       0.02078       0.04823   
absolute_criterion_change  4.405e-05     0.0001992   
absolute_params_change      0.002078      0.004823   

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)

The optimization will terminate because it reaches the maximum number of criterion evaluations. Of course, you could increase that number even further, but you can do better! Also: One million evaluations would not even be enough!